<a href="https://colab.research.google.com/github/dominiksakic/zero_to_hero/blob/main/basics_06_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- source: https://www.youtube.com/watch?v=kCc8FmEb1nY&t=126s

In [3]:
# get data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-12 23:51:16--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2025-07-12 23:51:16 (46.5 MB/s) - ‘input.txt.2’ saved [1115394/1115394]

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [23]:
# make decoder, encoder
stoi = {ch:i  for i, ch in enumerate(sorted(set(text)))}
itos = {i : ch for i, ch in enumerate(stoi)}

encoder = lambda s: [stoi[c] for c in s]
decoder = lambda l: ''.join([itos[i] for i in l])
print(encoder(text[:50]))
print(decoder(encoder(text[:50])))

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56]
First Citizen:
Before we proceed any further, hear


In [ ]:
# Tokenize data, and create test/val

In [ ]:
# create a baseline model/bigram

In [ ]:
# train the baseline

# Transformers


In [ ]:
# Gensis of Transformer
# Averaging past context with for loops
# Matrix multiplying as weighted aggregation
# Final version